
## Search through Profile properties summaries using Jinja2 and Python Modele

*USE Python 3.7 to maintain order of Json files*

- Fetch SD file from IG
- Spec_internals from IG Package.tgz
- search for element properties



*Note need a successful build to generate since based on ig output local file
alternatively use package.files to generate*

### import python modules including R4 fhirclient models

In [30]:
#from fhirclient.r4models import structuredefinition as SD
from fhir.resources import construct_fhir_element

from json import loads, dumps
from yaml import dump, load
from pathlib import Path
import copy
import pandas as pd

profile_map = {'http://hl7.org/fhir/StructureDefinition/Observation': 'Observation', 'http://hl7.org/fhir/StructureDefinition/vitalsigns':'FHIR Vital Signs Profile' }
ucum = 'http://unitsofmeasure.org'
out_path_csv = 'out/compare_obs_csv' 
out_path_excel = 'out/compare_obs_excel'
image_path = '/Users/ehaas/Documents/FHIR/US-Core/input/images'
#image_path = 'out/image'

### Create New DataFrame with My Custom Column Headers

In [31]:
empty_row = {
    'Profile Title': None,
    'Based On': None,  #US Core Profile or FHIR Observation Resource
    'Category Slice Codes': [], # URI|Code  
    'Code Binding': None,  #list patternCodeableConcept or binding with strength
    'Code Fixed Value': None, # list patternCodeableConcept or binding with strength
    'Effective Time Types': [], # list MS types
    'Value Types': [], # list MS types or slice by types
    'ValueQuantity Code Binding': None,  #binding with strength
    'ValueQuantity Code Fixed Value': None, # URI|fixedCode - always UCUM by value
    'Is Data Absent Reason': False, #True if MS
    'Is Component': False, #True if MS
    'Component Slice Codes': [], # URI|Code 
    'Component Slice Value Types': [], # list MS types
    #'Component Slice ValueQuantity Code Binding': None,  #binding with strength
    'Component Slice ValueQuantity Codes': [], # URI|fixedCode - always UCUM by value
    'Has Member References': [], #list if MS
    'Derived From References': [], #list if MS    
    }



df = pd.DataFrame(data={}, columns=empty_row.keys())
df

,Profile Title,Based On,Category Slice Codes,Code Binding,Code Fixed Value,Effective Time Types,Value Types,ValueQuantity Code Binding,ValueQuantity Code Fixed Value,Is Data Absent Reason,Is Component,Component Slice Codes,Component Slice Value Types,Component Slice ValueQuantity Codes,Has Member References,Derived From References


### Get file and return as dict

### Write to file

In [32]:
def save_new(f_name,data): # save in resources folder

    print('writing to file .......')
    #write_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/input/examples'
    write_path = 'test_resources'
    path = Path.cwd() / write_path  / f_name
    path.write_text(json.dumps(data, indent=2))

### Get all Profile from package file

  - create url:title mapping table



In [33]:
us_core_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev/package')
us_core_files = sorted(us_core_path.glob('StructureDefinition-*'))
for i in us_core_files:
    print(i.name)
    sd_dict = loads(i.read_text())
    profile_map[sd_dict['url']] = sd_dict['title']
profile_map

StructureDefinition-head-occipital-frontal-circumference-percentile.json
StructureDefinition-pediatric-bmi-for-age.json
StructureDefinition-pediatric-weight-for-height.json
StructureDefinition-us-core-allergyintolerance.json
StructureDefinition-us-core-birthsex.json
StructureDefinition-us-core-blood-pressure.json
StructureDefinition-us-core-bmi.json
StructureDefinition-us-core-body-height.json
StructureDefinition-us-core-body-temperature.json
StructureDefinition-us-core-body-weight.json
StructureDefinition-us-core-careplan.json
StructureDefinition-us-core-careteam.json
StructureDefinition-us-core-condition-encounter-diagnosis.json
StructureDefinition-us-core-condition-problems-health-concerns.json
StructureDefinition-us-core-diagnosticreport-lab.json
StructureDefinition-us-core-diagnosticreport-note.json
StructureDefinition-us-core-direct.json
StructureDefinition-us-core-documentreference.json
StructureDefinition-us-core-encounter.json
StructureDefinition-us-core-ethnicity.json
Structu

{'http://hl7.org/fhir/StructureDefinition/Observation': 'Observation',
 'http://hl7.org/fhir/StructureDefinition/vitalsigns': 'FHIR Vital Signs Profile',
 'http://hl7.org/fhir/us/core/StructureDefinition/head-occipital-frontal-circumference-percentile': 'US Core Pediatric Head Occipital-frontal Circumference Percentile Profile',
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age': 'US Core Pediatric BMI for Age Observation Profile',
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height': 'US Core Pediatric Weight for Height Observation Profile',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance': 'US Core AllergyIntolerance Profile',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex': 'US Core Birth Sex Extension',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-blood-pressure': 'US Core Blood Pressure Profile',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-bmi': 'US Core BMI 

### Loop through all sd and loop through all the elemenst and print out the element the element you want

filter on the element properties you want

In [34]:
for i in us_core_files:
    #print(i.name)
    sd_dict = loads(i.read_text())
    #print(sd_dict['type'])
    
    if sd_dict['type'] =="Observation":
        new_row = copy.deepcopy(empty_row)
        #print('='*80)
        #print(sd_dict['title'])
                
        #==== Title and BaseDefinition ====
        new_row['Profile Title'] = sd_dict['title']
        new_row['Based On'] = profile_map[sd_dict['baseDefinition']]
        # convert elemnt to hash
        ed_dict = {list(j.values())[0]:j for j in sd_dict['snapshot']['element']}
        # print(dump(ed_dict,sort_keys=False))
        
        #==== Category ====
        category_slices = [j for j in ed_dict if "category:" in j.split('.')[-1]]
        category_names = [ed_dict[k]['sliceName'] for k in category_slices]
        
        for k in category_slices:
            
            try:
              cat_system = ed_dict[k]['patternCodeableConcept']['coding'][0]['system']
              cat_code = ed_dict[k]['patternCodeableConcept']['coding'][0]['code']
            except KeyError:
              pass
            else:
              new_row['Category Slice Codes'].append(f'{cat_system}|{cat_code}')
            
            system_key = f'{k}.coding.system'
            code_key = f'{k}.coding.code'               
            try:
              cat_system = ed_dict[system_key]['fixedUri']
              cat_code = ed_dict[code_key]['fixedCode']
            except KeyError:
              pass
            else:
              new_row['Category Slice Codes'].append(f'{cat_system}|{cat_code}')

        #==== Code ====
        #pattern
        o_code = ed_dict['Observation.code']
        try:
            code_system = o_code['patternCodeableConcept']['coding'][0]['system']
            code_code = o_code['patternCodeableConcept']['coding'][0]['code']
        except KeyError:
            #binding
            binding = o_code['binding']['valueSet']
            strength = o_code['binding']['strength']
            new_row['Code Binding'] = f'{binding}({strength})'
        else:
            new_row['Code Fixed Value'] = f'{code_system}|{code_code}'
        
        #==== time types ====

        try:
            o_eff = ed_dict['Observation.effective[x]']
        except KeyError:
            pass
        else:
            for eff_type in o_eff['type']:
                try:
                    eff_type['extension'][0]['valueBoolean']==True
                except KeyError:
                    continue             
                else: 
                    new_row['Effective Time Types'].append(eff_type['code'])
       #==== values types  ====
    
        
       # MS by type
        try:
            val = ed_dict['Observation.value[x]']
        except KeyError:
            pass
        else:
            for val_type in val['type']:
                try:
                    assert val_type['extension'][0]['valueBoolean']==True
                except (KeyError, AssertionError):
                    continue             
                else: 
                    new_row['Value Types'].append(val_type['code']) 
                    
        # ===slice by type ===
        
        value_slices = [v for v in ed_dict if "value[x]:" in v.split('.')[-1]] 
        for k in value_slices:           
            try:
              val_type = ed_dict[k]['type'][0]['code']
            except KeyError:
              pass
            else:
              new_row['Value Types'].append(val_type)
              
            
        # ===Unit Codes ===
        # either binding or fixed or none       
        # 'ValueQuantity Code Binding': None,  #binding with strength
        # 'ValueQuantity Code Fixed Value': None, # URI|fixedCode - always UCUM by value
        try: 
            vq_code = ed_dict['Observation.value[x]:valueQuantity.code']
        except KeyError:
            pass
        else:
            try: 
                binding = vq_code['binding']['valueSet']
                strength = vq_code['binding']['strength']                         
            except KeyError:
                try:
                    new_row['ValueQuantity Code Fixed Value']=f'{ucum}|{vq_code["fixedCode"]}'
                except KeyError:
                    pass # no code          
            else:
                new_row['ValueQuantity Code Binding'] = f'{binding}({strength})'    
 
        # ===DAR a MS ===
        try:
            new_row['Is Data Absent Reason'] = ed_dict["Observation.dataAbsentReason"]["mustSupport"]
        except KeyError:
            pass
        
        # ===Component is MS ===
        try:
            new_row['Is Component'] = ed_dict["Observation.component"]["mustSupport"]
        except KeyError:
            pass
        else:
            
 
            # loop through all slices and find patterns
            comp_slices = [j for j in ed_dict if "component:" in j.split('.')[-1]]
            comp_names = [ed_dict[k]['sliceName'] for k in category_slices]

            for k in comp_slices:
                # 'Component Slice Codes': [], # URI|Code 
  
                comp_code_id = f'{k}.code'
                try:
                  comp_system = ed_dict[comp_code_id]['patternCodeableConcept']['coding'][0]['system']
                  comp_code = ed_dict[comp_code_id]['patternCodeableConcept']['coding'][0]['code']
                except KeyError:
                  pass
                else:
                  new_row['Component Slice Codes'].append(f'{cat_system}|{cat_code}')
                
                # 'Component Slice Value Types': None, # list MS types assume just first one for now
                # 'Component Slice ValueQuantity Codes': None, # URI|fixedCode - always UCUM by value
                comp_value_id = f'{k}.value[x].code'
                try:
                  val_type = ed_dict[comp_value_id]['type'][0]['code']
                  vq_code = ed_dict[comp_value_id]['fixedCode']
                except KeyError:
                  pass
                else:
                  new_row['Component Slice Value Types'].append(val_type)
                  new_row['Component Slice ValueQuantity Codes'].append(f'{ucum}|{vq_code}')
                            
                
        #  === Has Member refs ====

        try:
            has_m = ed_dict['Observation.hasMember']
        except KeyError:
            pass
        else:
            for i, ref in enumerate(has_m['type'][0]['targetProfile']):
                try:
                    assert has_m['type'][0]['_targetProfile'][i]['extension'][0]['valueBoolean']==True
                except (KeyError, AssertionError):
                    continue             
                else: 
                    new_row['Has Member References'].append(ref)
              
        
        # === derivedFrom refs ===
        
        try:
            der_from = ed_dict['Observation.derivedFrom']
        except KeyError:
            pass
        else:
            for i, ref in enumerate(der_from['type'][0]['targetProfile']):
                try:
                    assert der_from['type'][0]['_targetProfile'][i]['extension'][0]['valueBoolean']==True
                except (KeyError, AssertionError):
                    continue             
                else: 
                    new_row['Has Member References'].append(ref)
        

        # =====print(new_row) =====
        # print('='*80)
        df_length = len(df)
        df.loc[df_length] = new_row

    
df = df.mask(df.applymap(type).eq(list) & ~df.astype(bool))
df

,Profile Title,Based On,Category Slice Codes,Code Binding,Code Fixed Value,Effective Time Types,Value Types,ValueQuantity Code Binding,ValueQuantity Code Fixed Value,Is Data Absent Reason,Is Component,Component Slice Codes,Component Slice Value Types,Component Slice ValueQuantity Codes,Has Member References,Derived From References
0,US Core Pediatric Head Occipital-frontal Circu...,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|8289-1,[dateTime],[Quantity],None,http://unitsofmeasure.org|%,True,True,NaN,NaN,NaN,NaN,NaN
1,US Core Pediatric BMI for Age Observation Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|59576-9,[dateTime],[Quantity],None,http://unitsofmeasure.org|%,True,True,NaN,NaN,NaN,NaN,NaN
2,US Core Pediatric Weight for Height Observatio...,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|77606-2,[dateTime],[Quantity],None,http://unitsofmeasure.org|%,True,True,NaN,NaN,NaN,NaN,NaN
3,US Core Blood Pressure Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|85354-9,[dateTime],[Quantity],None,None,True,True,[http://terminology.hl7.org/CodeSystem/observa...,"[code, code]","[http://unitsofmeasure.org|mm[Hg], http://unit...",NaN,NaN
4,US Core BMI Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|39156-5,[dateTime],[Quantity],None,http://unitsofmeasure.org|kg/m2,True,True,NaN,NaN,NaN,NaN,NaN
5,US Core Body Height Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|8302-2,[dateTime],[Quantity],http://hl7.org/fhir/ValueSet/ucum-bodylength|4...,None,True,True,NaN,NaN,NaN,NaN,NaN
6,US Core Body Temperature Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|8310-5,[dateTime],[Quantity],http://hl7.org/fhir/ValueSet/ucum-bodytemp|4.0...,None,True,True,NaN,NaN,NaN,NaN,NaN
7,US Core Body Weight Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|29463-7,[dateTime],[Quantity],http://hl7.org/fhir/ValueSet/ucum-bodyweight|4...,None,True,True,NaN,NaN,NaN,NaN,NaN
8,US Core Head Circumference Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|9843-4,[dateTime],[Quantity],http://hl7.org/fhir/ValueSet/ucum-bodylength|4...,None,True,True,NaN,NaN,NaN,NaN,NaN
9,US Core Heart Rate Profile,US Core Vital Signs Profile,[http://terminology.hl7.org/CodeSystem/observa...,None,http://loinc.org|8867-4,[dateTime],[Quantity],None,http://unitsofmeasure.org|/min,True,True,NaN,NaN,NaN,NaN,NaN


In [35]:
print(f"printing {out_path_excel}/USCore_Obs_Compare.xlsx ....")
df.to_excel(f"{out_path_excel}/USCore_Obs_Compare.xlsx")
print(f"printing {out_path_csv}/USCore_Obs_Compare.csv ....")
df.to_csv(f"{out_path_csv}/USCore_Obs_Compare.csv")

printing out/compare_obs_excel/USCore_Obs_Compare.xlsx ....
printing out/compare_obs_csv/USCore_Obs_Compare.csv ....


In [36]:
%%bash -s "$out_path_excel" "$out_path_csv" "$image_path"
echo "================================================================="
echo "===zip up csv and excel files and put in==="
echo "===$3/argo-compare.zips file for downloads==="
echo "================================================================="
zip -j $3/argo-obs-compare-excel.zip $1/*.*
zip -j $3/argo-obs-compare-csv.zip $2/*.*

===zip up csv and excel files and put in===
===/Users/ehaas/Documents/FHIR/US-Core/input/images/argo-compare.zips file for downloads===
  adding: README.md (deflated 60%)
  adding: USCore_Obs_Compare.xlsx (deflated 8%)
  adding: README.md (deflated 60%)
  adding: USCore_Obs_Compare.csv (deflated 83%)
